In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata
import copy

In [2]:
def apply_rule(pattern,sen):
#     sen_list = re.split(r'([\,\.\?])',sen)
#     for part in sen_list:
#         if not part == ',' or part == '.' or part == '?':
#             sen = '.' + part + '.'
    sen = '.' + sen + '.'
    p1 = pattern.split('>>')[0]#字符串
    p2 = pattern.split('>>')[1].split()
    if re.findall(p1,sen):
        result = re.findall(p1,sen)[0]
        same_str = []
        if isinstance(result,tuple):
            for i in range(len(result)):
                same_str.append(result[i])
        if isinstance(result,str):
            same_str.append(result)
        answer=[]
        for item in p2:
            if '$@_' in item:
                nu = int(re.findall('[0-9]+',item)[0])
                curword = same_str[nu]
                if not curword.endswith(' '):
                     curword = curword + ' '
                if(curword.startswith(' ')):
                    curword = curword[1:]

                answer.append(curword)
            else:

                if not item.endswith(' '):
                     item = item + ' '
                if(item.startswith(' ')):
                    item = item[1:]
                answer.append(item)
#             answer_list.append(answer)
        return  (''.join(answer)[:-1])
#         else:
#             answer_list.append(part)
#     answer_list.append(''.join(answer_list)[:-1])
    else:
        return ""

In [3]:
df1 = pd.read_csv('quora.csv',encoding='gbk')
print(len(df1))
df1.head()

292007


,question1,question2
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,What would happen if the Indian government sto...,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,How can Internet speed be increased by hacking...,How can I increase the speed of my internet co...
3,What keeps childern active and far from phone ...,Should I buy tiago ?
4,What should I do to be a great geologist ?,How can I be a good geologist ?


In [286]:
#rule 生成

In [29]:
rule=[]
for i in range(len(df1)):
    sentence1 = df1['question1'][i].lower().split()
#     sentence1 = 'what should i do if i want go home ?'.split()
    sentence1.pop()
    sentence2 = df1['question2'][i].lower().split()
#     sentence2 = 'what can i do if i want go home ?'.split()
    sentence2.pop()

    margin = 1
    flag = True
    masked = '%@%'
    
    r1 = ''   
    r2 = [x for x in sentence2]
    
    p = "\$@_\d+|\d+\$@_"#filter p2 that contains &@_number    
    for item in r2:
        if re.findall(p,item):
            flag = False
            break
    if flag:    
        sentence1_len = len(sentence1)
        sentence2_len = len(sentence2)
        id_dollar =0
        count = 0
        j = 0
        while j < len(sentence1):
            try:
                #对相同单词处理
                word = sentence1[j]
                id = sentence2.index(word)
                maxlen = 0
                xval = 0
                #计算从当前单词出发的相同字符子串长度maxlen
                while sentence1[j+xval] == sentence2[id+xval]:
                    maxlen = maxlen + 1 
                    xval = xval + 1
                    if j+xval >= sentence1_len or id+xval >=sentence2_len:
                        break
                        
                if maxlen > 1:
                    index = r2.index(word)
                    while xval > 0:
                        del r2[index]
                        sentence2[id+maxlen-xval] = masked
                        xval = xval - 1
                    #修改规则r1
                    cur_word1 =r'(.+)'
                    r1+=cur_word1
                    #修改规则r2
                    cur_word2 = '$@_{}'.format(id_dollar)
                    id_dollar +=1
                    r2.insert(index,cur_word2)
                    #修改循环变量j
                    j = j + maxlen 
                elif maxlen == 1:
                    #修改规则r1
                    cur_word1 =r'(\S+)'
                    r1+=cur_word1
                    #修改规则r2
                    index = r2.index(word)
                    cur_word2 = '$@_{}'.format(id_dollar) 
                    id_dollar +=1
                    r2[index] = cur_word2
                    #修改原句sentence2
                    sentence2[id] = masked
                    #修改循环变量j
                    j = j + 1
            except:
                #对不同单词处理
                cur_word1 = word           
                pattern = ".*[\\\|\*\[\]\{\}\|\(\)\+\.\?\$\^\/].*"
                if re.findall(pattern,cur_word1):            
                    flag=False
                    break    
                #修改规则r1
                r1+=cur_word1
                count = count + 1 
                #修改循环变量j
                j = j + 1
                
                
            if j<len(sentence1):
                r1+='\s' 
        r2_len = len(r2)
        if count < margin or (r2_len-id_dollar-1)<margin:#r1、r2中至少有margin个单词以上
            flag = False  
        if flag:
            r1 = '[\,\.\?]' + r1 + '[\,\.\?]'
            rule.append(''.join(r1) +'>>'+ ' '.join(r2))
#     if i > 10:
#         break

In [30]:
for i in range(len(df1)):
    sentence1 = df1['question2'][i].lower().split()
    sentence1.pop()
    sentence2 = df1['question1'][i].lower().split()
    sentence2.pop()

    margin = 1
    flag = True
    masked = '%@%'
    
    r1 = ''   
    r2 = [x for x in sentence2]
    
    p = "\$@_\d+|\d+\$@_"#filter p2 that contains &@_number    
    for item in r2:
        if re.findall(p,item):
            flag = False
            break
    if flag:    
        sentence1_len = len(sentence1)
        sentence2_len = len(sentence2)
        id_dollar =0
        count = 0
        j = 0
        while j < len(sentence1):
#         for j in len(sentence1):
#             j = sentence1.index(word)
            try:
                #对相同单词 处理
                word = sentence1[j]
                id = sentence2.index(word)
                maxlen = 0
                xval = 0
                #计算从当前单词出发的相同字符子串长度maxlen
                while sentence1[j+xval] == sentence2[id+xval]:
                    maxlen = maxlen + 1 
                    xval = xval + 1
                    if j+xval >= sentence1_len or id+xval >=sentence2_len:
                        break
                        
                if maxlen > 1:
                    index = r2.index(word)
                    while xval > 0:
                        del r2[index]
                        sentence2[id+maxlen-xval] = masked
                        xval = xval - 1
                    #修改规则r1
                    cur_word1 =r'(.+)'
                    r1+=cur_word1
                    #修改规则r2
                    cur_word2 = '$@_{}'.format(id_dollar)
                    id_dollar +=1
                    r2.insert(index,cur_word2)
                    #修改循环变量j
                    j = j + maxlen 
                elif maxlen == 1:
                    #修改规则r1
                    cur_word1 =r'(.+)'
                    r1+=cur_word1
                    #修改规则r2
                    index = r2.index(word)
                    cur_word2 = '$@_{}'.format(id_dollar) 
                    id_dollar +=1
                    r2[index] = cur_word2
                    #修改原句sentence2
                    sentence2[id] = masked
                    #修改循环变量j
                    j = j + 1
            except:
                #对不同单词处理
                cur_word1 = word           
                pattern = ".*[\\\|\*\[\]\{\}\|\(\)\+\.\?\$\^\/].*"
                if re.findall(pattern,cur_word1):            
                    flag=False
                    break    
                #x修改规则r1
                r1+=cur_word1
                count = count + 1 
                #修改循环变量j
                j = j + 1
                
                
            if j<len(sentence1):
                r1+='\s' 
        r2_len = len(r2)
        if count < margin or (r2_len-id_dollar-1)<margin:#r1、r2中至少有margin个单词以上
            flag = False  
        if flag:
            r1 = '[\,\.\?]' + r1 + '[\,\.\?]'
            rule.append(''.join(r1) +'>>'+ ' '.join(r2))

In [31]:
# rule = list(set(rule))
print(len(rule))
rule[:5]

450656


['[\\,\\.\\?](\\S+)\\swould\\shappen\\sif\\s(\\S+)\\sindian\\sgovernment\\sstole\\sthe\\s(.+)\\sback[\\,\\.\\?]>>$@_0 is $@_1 story of $@_2',
 '[\\,\\.\\?](.+)\\s(\\S+)\\s(\\S+)\\sbe\\sincreased\\sby\\shacking\\sthrough\\sdns[\\,\\.\\?]>>$@_0 i increase the $@_2 of my $@_1 connection while using a vpn',
 '[\\,\\.\\?]what\\skeeps\\schildern\\sactive\\sand\\sfar\\sfrom\\sphone\\sand\\svideo\\sgames[\\,\\.\\?]>>should i buy tiago',
 '[\\,\\.\\?]what\\sshould\\s(\\S+)\\sdo\\sto\\s(.+)\\sgreat\\s(\\S+)[\\,\\.\\?]>>how can $@_0 $@_1 good $@_2',
 '[\\,\\.\\?](.+)\\s"&"\\s(.+)\\s"and"[\\,\\.\\?]>>$@_0 シ $@_1 し']

In [176]:
#测试

In [ ]:
sen = 'I want to learn coding how should I start it'.lower()
print("rule[0]",rule[0])
print(apply_rule(rule[0],sen))

In [ ]:
#将新生成规则与人工筛选规则合并

In [41]:
with open('rule.txt','r') as f:
    ruless = f.read().split('\n')
    ruless.pop()
print(ruless[:3])

['[\\,\\.\\?]how\\sdo\\si\\s(.+)[\\,\\.\\?]>>what is the best way to $@_0', '[\\,\\.\\?]how\\sdo\\si\\s(.+)[\\,\\.\\?]>>what are some ways to $@_0', "[\\,\\.\\?]how\\sdo\\si\\s(.+)[\\,\\.\\?]>>what 's the best way to $@_0"]


In [33]:
rule1 = rule + ruless

In [34]:
print(len(rule1))

450801


In [27]:
#rule 排序

In [35]:
dic = {}
for r in rule1:
    if r in dic:
        dic[r]+=1
    else:
        dic[r]=1
sort_rule1 = sorted(dic.items(), key=lambda e:e[1], reverse=True)

In [36]:
rules=[]
count=[]
for i in range(len(sort_rule1)):
    if sort_rule1[i][1] > 1:
        rules.append(sort_rule1[i][0])
        count.append(sort_rule1[i][1])

In [37]:
print(len(rules))
print(count[:400])

3256
[148, 138, 105, 94, 52, 52, 50, 50, 43, 43, 42, 36, 29, 28, 28, 27, 27, 26, 26, 25, 25, 24, 24, 24, 23, 23, 23, 22, 22, 22, 22, 22, 22, 21, 21, 20, 20, 20, 20, 20, 20, 20, 19, 18, 18, 18, 18, 18, 17, 17, 17, 17, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 14, 14, 14, 14, 14, 14, 14, 14, 14, 13, 13, 13, 13, 13, 13, 13, 12, 12, 12, 12, 12, 12, 12, 12, 12, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 

In [ ]:
#rule 筛选

In [38]:
ruless = []
for rule in rules:
    count0 = 0
    
    p1 = rule.split('>>')[1].split()
    p2 = rule.split('>>')[1].split()
    
    pattern1 = "\$\d+"
    pattern2 = "\d+\$"
    
    flag_filter = False
    
    for item1 in p1:
        list_test1 = re.findall(pattern1,item1)
        list_test2 = re.findall(pattern2,item1)
        
        if list_test1 or list_test2:
            flag_filter = True

    for item2 in p2:
        list_test1 = re.findall(pattern1,item2)
        list_test2 = re.findall(pattern2,item2)
        
        if list_test1 or list_test2:
                flag_filter = True
       
        if '$' == item2:
            flag_filter = True
               
    
        if '$@_' in item2:
            count0 = count0 + 1            
    if flag_filter:
        continue
        
    if count0 > 0:
        ruless.append(rule)
        


In [39]:
print(len(ruless))
print(ruless[:10])

3256
['[\\,\\.\\?]how\\sdo\\si\\s(.+)[\\,\\.\\?]>>what is the best way to $@_0', '[\\,\\.\\?]what\\sis\\sthe\\sbest\\sway\\sto\\s(.+)[\\,\\.\\?]>>how do i $@_0', '[\\,\\.\\?]how\\scan\\si\\s(.+)[\\,\\.\\?]>>what is the best way to $@_0', '[\\,\\.\\?]what\\sis\\sthe\\sbest\\sway\\sto\\s(.+)[\\,\\.\\?]>>how can i $@_0', '[\\,\\.\\?]how\\sdo\\syou\\s(.+)[\\,\\.\\?]>>what is the best way to $@_0', '[\\,\\.\\?](.+)\\ssome\\sgood\\s(.+)[\\,\\.\\?]>>$@_0 the best $@_1', '[\\,\\.\\?](.+)\\sthe\\sbest\\s(.+)[\\,\\.\\?]>>$@_0 some good $@_1', '[\\,\\.\\?]what\\sis\\sthe\\sbest\\sway\\sto\\s(.+)[\\,\\.\\?]>>how do you $@_0', '[\\,\\.\\?]can\\s(.+)[\\,\\.\\?]>>how do $@_0', '[\\,\\.\\?](.+)\\scan\\si\\s(.+)[\\,\\.\\?]>>$@_0 do you $@_1']


In [40]:
with open('rule_filter_test.txt','w') as f:
    for r in ruless:
        try:
            f.write(r)
            f.write('\n')
        except:
            r = r.encode("gb18030")
            r = str(r) 

In [241]:
with open('rule_filter_test.txt','r') as f:
    ruless = f.read().split('\n')
    ruless.pop()
print(ruless[:10])

['[\\,\\.\\?]how\\sdo\\si\\s(.+)[\\,\\.\\?]>>what is the best way to $@_0', '[\\,\\.\\?]how\\scan\\si\\s(.+)[\\,\\.\\?]>>what is the best way to $@_0', '[\\,\\.\\?]how\\sdo\\syou\\s(.+)[\\,\\.\\?]>>what is the best way to $@_0', '[\\,\\.\\?]how\\sdo\\si\\s(.+)[\\,\\.\\?]>>what are the best ways to $@_0', '[\\,\\.\\?]how\\scan\\si\\s(.+)[\\,\\.\\?]>>what are the best ways to $@_0', '[\\,\\.\\?]how\\sdo\\syou\\s(.+)[\\,\\.\\?]>>what are some ways to $@_0', "[\\,\\.\\?]how\\sdo\\si\\s(.+)[\\,\\.\\?]>>what 's the best way to $@_0", '[\\,\\.\\?]how\\scan\\si\\s(.+)[\\,\\.\\?]>>what are some ways to $@_0', '[\\,\\.\\?]how\\sdo\\si\\s(.+)[\\,\\.\\?]>>what are some ways to $@_0', "[\\,\\.\\?]how\\scan\\si\\s(.+)[\\,\\.\\?]>>what 's the best way to $@_0"]


In [243]:
sen_fail = []
count =[]
for i in range(len(df1)):
#     sen = 'how do i go home'
    count.append(0)
    sen=df1['question1'][i].lower()
    sen = sen.replace(' ?','')
    flag1_ = True
    for j in range(len(ruless)):
        answer = apply_rule(ruless[j],sen)
        if answer:
            flag1_ = False
            print('\n','sen',sen,'\n','rule',ruless[j],'\n','answer',answer,'\n')
            count[i] = count[i] + 1
    if flag1_:
        sen_fail.append(df1['question1'][i])
    if i > 5:
        break


 sen what is the step by step guide to invest in share market 
 rule [\,\.\?]what\sis\sthe\s(.+)[\,\.\?]>>how would you take over $@_0 
 answer how would you take over step by step guide to invest in share market 


 sen what is the step by step guide to invest in share market 
 rule [\,\.\?]what\sis\sthe\s(\S+)\s(.+)[\,\.\?]>>which $@_1 would be $@_0 for seo services 
 answer which by step guide to invest in share market would be step for seo services 


 sen what is the step by step guide to invest in share market 
 rule [\,\.\?]what\sis\sthe\s(\S+)\s(\S+)\s(.+)[\,\.\?]>>which $@_1 will be $@_0 $@_2 now 
 answer which by will be step step guide to invest in share market now 


 sen what is the step by step guide to invest in share market 
 rule [\,\.\?]what\sis\sthe\s(.+)[\,\.\?]>>how do i apply for a job in $@_0 especially for chennaiyin fc 
 answer how do i apply for a job in step by step guide to invest in share market especially for chennaiyin fc 


 sen what is the step by step


 sen what would happen if the indian government stole the kohinoor (koh-i-noor) diamond back 
 rule [\,\.\?]what\swould\shappen\s(\S+)\s(\S+)\s(.+)\s(\S+)[\,\.\?]>>how will $@_3 's economy be affected $@_0 india $@_2 $@_1 
 answer how will back 's economy be affected if india indian government stole the kohinoor (koh-i-noor) diamond the 


 sen what would happen if the indian government stole the kohinoor (koh-i-noor) diamond back 
 rule [\,\.\?]what\swould\shappen\sif\s(\S+)\s(.+)\s(\S+)[\,\.\?]>>how could $@_0 north $@_1 be $@_2 
 answer how could the north indian government stole the kohinoor (koh-i-noor) diamond be back 


 sen what would happen if the indian government stole the kohinoor (koh-i-noor) diamond back 
 rule [\,\.\?](.+)\sthe\s(.+)\sgovernment\s(.+)\sthe\s(.+)\s(.+)[\,\.\?]>>$@_0 did $@_1 $@_2 $@_3 rupees $@_4 and not 100 rupee notes 
 answer what would happen if did indian stole kohinoor (koh-i-noor) diamond rupees back and not 100 rupee notes 


 sen what would happ


 sen what should i do to be a great geologist 
 rule [\,\.\?](\S+)\sshould\si\sdo\s(\S+)\s(.+)[\,\.\?]>>$@_0 will $@_2 be like $@_1 you 
 answer what will be a great geologist be like to you 


 sen what should i do to be a great geologist 
 rule [\,\.\?]what\sshould\si\sdo\sto\s(.+)[\,\.\?]>>how can you $@_0 in the philippines 
 answer how can you be a great geologist in the philippines 


 sen what should i do to be a great geologist 
 rule [\,\.\?](\S+)\sshould\si\sdo\s(.+)\s(\S+)[\,\.\?]>>$@_0 is the best way $@_1 body $@_2 
 answer what is the best way to be a great body geologist 


 sen what should i do to be a great geologist 
 rule [\,\.\?]what\s(\S+)\si\sdo\s(\S+)\s(.+)[\,\.\?]>>anybody wants know $@_1 know how you $@_0 $@_2 a month 
 answer anybody wants know to know how you should be a great geologist a month 


 sen what should i do to be a great geologist 
 rule [\,\.\?]what\s(\S+)\si\sdo\sto\s(.+)[\,\.\?]>>$@_0 you $@_1 in 4 days 
 answer should you be a great geologist


 sen what should i do to be a great geologist 
 rule [\,\.\?]what\s(.+)\si\sdo\sto\s(.+)[\,\.\?]>>$@_0 you $@_1 in 6 months 
 answer should you be a great geologist in 6 months 


 sen what should i do to be a great geologist 
 rule [\,\.\?]what\sshould\s(.+)\sdo\sto\s(.+)[\,\.\?]>>how can $@_0 $@_1 at 16 
 answer how can i be a great geologist at 16 


 sen what should i do to be a great geologist 
 rule [\,\.\?]what\sshould\s(.+)\sdo\sto\s(.+)[\,\.\?]>>how can $@_0 $@_1 if someone instaigates me 
 answer how can i be a great geologist if someone instaigates me 


 sen what should i do to be a great geologist 
 rule [\,\.\?]what\sshould\s(.+)\sdo\sto\s(.+)\s(.+)[\,\.\?]>>how can $@_0 $@_1 communication skill and $@_2 proficiency 
 answer how can i be a great communication skill and geologist proficiency 


 sen what should i do to be a great geologist 
 rule [\,\.\?]what\sshould\s(.+)\s(.+)\sto\s(.+)[\,\.\?]>>how $@_1 $@_0 $@_2 in the afternoon 
 answer how do i be a great geologist 


 sen what should i do to be a great geologist 
 rule [\,\.\?](.+)\sshould\si\sdo\s(.+)\s(.+)[\,\.\?]>>$@_0 is the best way $@_1 white hat $@_2 
 answer what is the best way to be a great white hat geologist 


 sen what should i do to be a great geologist 
 rule [\,\.\?]what\s(.+)\s(.+)\sdo\sto\s(.+)\s(.+)\s(.+)[\,\.\?]>>how $@_1 $@_0 $@_2 for a $@_3 time $@_4 
 answer how i should be a for a great time geologist 


 sen what should i do to be a great geologist 
 rule [\,\.\?](.+)\sshould\si\s(.+)\s(.+)\sa\s(.+)[\,\.\?]>>$@_0 does one $@_1 the $@_2 b $@_3 examination 
 answer what does one do to the be b great geologist examination 


 sen what should i do to be a great geologist 
 rule [\,\.\?]what\sshould\s(.+)\s(.+)\sto\s(.+)[\,\.\?]>>how can $@_0 $@_2 but also $@_1 healthy 
 answer how can i be a great geologist but also do healthy 


 sen what should i do to be a great geologist 
 rule [\,\.\?]what\sshould\si\sdo\sto\s(.+)[\,\.\?]>>how can you $@_0 without having money 
 answer h

...


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)\s(.+)\s(.+)[\,\.\?]>>what 's $@_1 best way to $@_0 a $@_2 
 answer what 's free best way to hack motorola dcx3400 for a internet 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)[\,\.\?]>>what are some good websites to $@_0 
 answer what are some good websites to hack motorola dcx3400 for free internet 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)\s(.+)[\,\.\?]>>what 's the best recipe to $@_0 your own $@_1 
 answer what 's the best recipe to hack motorola dcx3400 for free your own internet 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)\s(.+)[\,\.\?]>>can $@_1 $@_0 by reading and listening 
 answer can internet hack motorola dcx3400 for free by reading and listening 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)\s(.+)[\,\.\?]>>what is th

 answer what is the easiest way to hack motorola dcx3400 for free a lot of time or internet 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)\s(.+)\s(.+)[\,\.\?]>>what are some tips to $@_0 $@_2 $@_1 comprehension 
 answer what are some tips to hack motorola dcx3400 for internet free comprehension 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)[\,\.\?]>>is it possible for a 33 year old felon to $@_0 
 answer is it possible for a 33 year old felon to hack motorola dcx3400 for free internet 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)\s(.+)[\,\.\?]>>should some boys learn to $@_0 their $@_1 
 answer should some boys learn to hack motorola dcx3400 for free their internet 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)\sfor\s(.+)[\,\.\?]>>what is the best $@_1 site to $@_0 from scratch 
 answer what is the best free internet si


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)\s(.+)[\,\.\?]>>what 's the easiest way to $@_0 pc $@_1 
 answer what 's the easiest way to hack motorola dcx3400 for free pc internet 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)\s(.+)[\,\.\?]>>what is the best way to $@_0 and also distribute a $@_1 
 answer what is the best way to hack motorola dcx3400 for free and also distribute a internet 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)\s(.+)\s(.+)[\,\.\?]>>is there any way by which we can $@_0 all $@_1 and sms $@_2 
 answer is there any way by which we can hack motorola dcx3400 for all free and sms internet 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\s(.+)\si\s(.+)\sfor\s(.+)[\,\.\?]>>what would someone need to $@_0 to $@_1 to $@_2 old 
 answer what would someone need to do to hack motorola dcx3400 to free internet old 


 sen how 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)\s(.+)[\,\.\?]>>what are some of the best ways to $@_0 our $@_1 
 answer what are some of the best ways to hack motorola dcx3400 for free our internet 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)[\,\.\?]>>what are the best ways to $@_0 of your customers 
 answer what are the best ways to hack motorola dcx3400 for free internet of your customers 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)[\,\.\?]>>whats the most effective way/ solution to $@_0 
 answer whats the most effective way/ solution to hack motorola dcx3400 for free internet 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)\s(.+)[\,\.\?]>>what is the best way to $@_0 everything about $@_1 
 answer what is the best way to hack motorola dcx3400 for free everything about internet 


 sen how do i hack motorola dcx3400 for


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)\s(.+)[\,\.\?]>>can we $@_0 our business $@_1 
 answer can we hack motorola dcx3400 for free our business internet 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)\s(.+)\s(.+)\s(.+)[\,\.\?]>>what are $@_1 of the best ways to $@_0 $@_3 in a $@_2 way 
 answer what are for of the best ways to hack motorola dcx3400 internet in a free way 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)\s(.+)[\,\.\?]>>what are some of the best ways to $@_0 a $@_1 on a call 
 answer what are some of the best ways to hack motorola dcx3400 for free a internet on a call 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(.+)[\,\.\?]>>what is a recognized institute to $@_0 
 answer what is a recognized institute to hack motorola dcx3400 for free internet 


 sen how do i hack motorola dcx3400 for free internet 
 rule

In [244]:
print(sen_fail)
for item in count:
    print(item)

['What keeps childern active and far from phone and video games ?']
35
4
8
0
137
7
394


In [88]:
word = 'dsafa'
sen = 'dsafs dsafa dsaf'
sen = sen.split()
id = sen.index(word)
del sen[id]
del sen[id]
print(sen)
sen.insert(1,'fdfds')
print(sen)
sen = '123456'

['dsafs']
['dsafs', 'fdfds']


In [94]:
i=0
while i < 6:
    print(i)
    if i > 2:
        i = 5
    print('fd',i)
    i = i + 1 
    print('df',i)

0
fd 0
df 1
1
fd 1
df 2
2
fd 2
df 3
3
fd 5
df 6


In [249]:
with open('rule_filter_test.txt','r') as f:
    ruless = f.read().split('\n')
    ruless.pop()
print(ruless[:10])

['[\\,\\.\\?]how\\sdo\\si\\s(.+)[\\,\\.\\?]>>what is the best way to $@_0', '[\\,\\.\\?]how\\scan\\si\\s(.+)[\\,\\.\\?]>>what is the best way to $@_0', '[\\,\\.\\?]how\\sdo\\syou\\s(.+)[\\,\\.\\?]>>what is the best way to $@_0', '[\\,\\.\\?]how\\sdo\\si\\s(.+)[\\,\\.\\?]>>what are the best ways to $@_0', '[\\,\\.\\?]how\\scan\\si\\s(.+)[\\,\\.\\?]>>what are the best ways to $@_0', '[\\,\\.\\?]how\\sdo\\syou\\s(.+)[\\,\\.\\?]>>what are some ways to $@_0', "[\\,\\.\\?]how\\sdo\\si\\s(.+)[\\,\\.\\?]>>what 's the best way to $@_0", '[\\,\\.\\?]how\\scan\\si\\s(.+)[\\,\\.\\?]>>what are some ways to $@_0', '[\\,\\.\\?]how\\sdo\\si\\s(.+)[\\,\\.\\?]>>what are some ways to $@_0', "[\\,\\.\\?]how\\scan\\si\\s(.+)[\\,\\.\\?]>>what 's the best way to $@_0"]


In [253]:
rules = []
for rule in ruless:
    count0 = 0
    
    p1 = rule.split('>>')[1].split()
    p2 = rule.split('>>')[1].split()
    
    pattern1 = "\$\d+"
    pattern2 = "\d+\$"
    pattern3 = '\d+'
    pattern4 = 'way'
    
    flag_filter = False
    
    for item1 in p1:
        list_test1 = re.findall(pattern1,item1)
        list_test2 = re.findall(pattern2,item1)
        list_test3 = re.findall(pattern3,item1)
        list_test4 = re.findall(pattern4,item1)
        if list_test1 or list_test2 or list_test3 or list_test4:
            flag_filter = True

    for item2 in p2:
        list_test1 = re.findall(pattern1,item2)
        list_test2 = re.findall(pattern2,item2)
        list_test3 = re.findall(pattern3,item1)
        list_test4 = re.findall(pattern4,item1)
        if list_test1 or list_test2 or list_test3 or list_test4:
                flag_filter = True
       
        if '$' == item2:
            flag_filter = True
               
    
        if '$@_' in item2:
            count0 = count0 + 1            
    if flag_filter:
        continue
        
    if count0 > 0:
        rules.append(rule)
        


In [254]:
print(len(rules))

0


In [262]:
with open('rule_filter_test.txt','r') as f:
    ruless = f.read().split('\n')
    ruless.pop()
print(ruless[:10])
print(len(ruless))

['[\\,\\.\\?]how\\sdo\\si\\s(.+)[\\,\\.\\?]>>what is the best way to $@_0', '[\\,\\.\\?]how\\scan\\si\\s(.+)[\\,\\.\\?]>>what is the best way to $@_0', '[\\,\\.\\?]how\\sdo\\syou\\s(.+)[\\,\\.\\?]>>what is the best way to $@_0', '[\\,\\.\\?]how\\sdo\\si\\s(.+)[\\,\\.\\?]>>what are the best ways to $@_0', '[\\,\\.\\?]how\\scan\\si\\s(.+)[\\,\\.\\?]>>what are the best ways to $@_0', '[\\,\\.\\?]how\\sdo\\syou\\s(.+)[\\,\\.\\?]>>what are some ways to $@_0', "[\\,\\.\\?]how\\sdo\\si\\s(.+)[\\,\\.\\?]>>what 's the best way to $@_0", '[\\,\\.\\?]how\\scan\\si\\s(.+)[\\,\\.\\?]>>what are some ways to $@_0', '[\\,\\.\\?]how\\sdo\\si\\s(.+)[\\,\\.\\?]>>what are some ways to $@_0', "[\\,\\.\\?]how\\scan\\si\\s(.+)[\\,\\.\\?]>>what 's the best way to $@_0"]
243681


In [263]:
sen  ='[\,\.\?]what\sis\sthe\s(\S+)\s(.+)[\,\.\?]>>which $@_1 would be $@_0 for seo services'
count = 0
for i in ruless:
    if sen == i:
        count = count + 1
print(count)

1


In [ ]:
"what is the name of joe"
"what is the joe 's name"
x y of z
x z 's y
"what x was the name of joe"
"what x was the joe 's name"
